In [ ]:
import cv2
import numpy as np
from pynput.mouse import Button, Controller
import wx
mouse=Controller()

app=wx.App(False)
(sx,sy)=wx.GetDisplaySize()
(camx,camy)=(320,240)

lowerBound=np.array([33,80,40])
upperBound=np.array([102,255,255])
#lowerBound = np.array([0,255,255])
#upperBound = np.array([179,255,255])
cam= cv2.VideoCapture(0)
cam.set(3,camx)
cam.set(4,camy)
kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))
pinchFlag=0

while True:
    k = cv2.waitKey(1) & 0xFF
    if k==27:
        break
    else:
        ret, img=cam.read()
       # img=cv2.resize(img,(340,220))
        img = cv2.flip(img,1)
        #convert BGR to HSV
        imgHSV= cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
        # create the Mask
        mask=cv2.inRange(imgHSV,lowerBound,upperBound)
        #morphology
        maskOpen=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernelOpen)
        maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

        maskFinal=maskClose
        conts,h=cv2.findContours(maskFinal.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

        if(len(conts)==1):
            mouse.release(Button.left)
            x,y,w,h=cv2.boundingRect(conts[0])
            
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            cx=x+w/2
            cy=y+h/2
            cv2.circle(img,(cx,cy),(w+h)/4,(0,0,255),2)
            mouseLoc=(sx-(cx*sx/camx), cy*sy/camy)
            mouse.position=mouseLoc 
            while mouse.position!=mouseLoc:
                pass
        cv2.imshow("cam",img)
        cv2.waitKey(5)